In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
from pprint import pprint
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
%matplotlib inline
# %matplotlib notebook
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree


In [2]:
def PrepareData(train_data_frm):
    #print(train_data_frm.columns.values)
    train_data_frm = DropColumns(train_data_frm)
    train_data_frm = FillMissingValues(train_data_frm)
    df = train_data_frm
    nan_rows = df[df.isnull().T.any().T]
    return train_data_frm

#train_data= pd.read_csv("train.csv")
#PrepareData(train_data)

In [3]:
def FillMissingValues(train_data_frm):
    
    fill_col=['YrSold','MoSold','MiscVal','PoolArea','ScreenPorch',
              '3SsnPorch','EnclosedPorch','OpenPorchSF','WoodDeckSF',
              'GarageArea','GarageCars','GarageYrBlt','Fireplaces',
              'TotRmsAbvGrd','Kitchen','Bedroom','LotFrontage','LotArea',
              'YearBuilt','YearRemodAdd','MasVnrArea','BsmtFinSF1','BsmtFinSF2',
              'BsmtUnfSF','TotalBsmtSF','1stFlrSF','2ndFlrSF','LowQualFinSF',
              'GrLivArea','BsmtFullBath','BsmtHalfBath','FullBath','HalfBath']
    for val in train_data_frm.columns.values:
        if(val in fill_col):
            train_data_frm[val] = train_data_frm[val].fillna((train_data_frm[val].mean()))
        else:
            train_data_frm[val] = train_data_frm[val].fillna((train_data_frm[val].mode()[0]))      
    return train_data_frm

In [4]:
def DropColumns(train_data_frm):
    train_data_frm = train_data_frm.drop('Alley', axis=1)
    train_data_frm = train_data_frm.drop('PoolQC', axis=1)
    train_data_frm = train_data_frm.drop('Fence', axis=1)
    train_data_frm = train_data_frm.drop('MiscFeature', axis=1)
    train_data_frm = train_data_frm.drop('Id', axis=1)

    return train_data_frm

def check_categorical(column):
        fill_col=['YrSold','MoSold','MiscVal','PoolArea','ScreenPorch',
              '3SsnPorch','EnclosedPorch','OpenPorchSF','WoodDeckSF',
              'GarageArea','GarageCars','GarageYrBlt','Fireplaces',
              'TotRmsAbvGrd','Kitchen','Bedroom','LotFrontage','LotArea',
              'YearBuilt','YearRemodAdd','MasVnrArea','BsmtFinSF1','BsmtFinSF2',
              'BsmtUnfSF','TotalBsmtSF','1stFlrSF','2ndFlrSF','LowQualFinSF',
              'GrLivArea','BsmtFullBath','BsmtHalfBath','FullBath','HalfBath']
        if(column in fill_col):
            return False
        return True 


In [5]:
def FillWithUniqueCat(train_data,Col):
    # print( " in FillWithUniqueCat ")
    df  = train_data[Col]
    lis = df.values
    mylist = list(set(lis))
    return mylist

In [6]:
def FillWithUniqueContinuous(train_data,col):
    # print(" in FillWithUniqueContinuous ")
    df  = train_data[col]
    lis = df.values
    lis = list(set(lis))
    val = 0
    lis1=[]
    if(len(lis) == 1):
        return lis
    for i in range(len(lis)-1):
        lis1.append((lis[i]+lis[i+1])/2)
    return  lis1   

In [7]:
class dtree_regressor :
    
    def __init__(self):
        name=""
        
    sub_tree =  dict()
    
    def train(self,filename):
        train_data = pd.read_csv(filename)
        train_data = PrepareData(train_data)
        self.sub_tree = self.DecisionTree(train_data, counter = 0,
                     min_samples = 2, max_depth = 5)
        
        
    def test(self,filename):
        test_data = pd.read_csv(filename)
        test_data = PrepareData(test_data)
        return [self.classify_example(row, self.sub_tree) for i, row in test_data.iterrows()]
    
    def make_dictionary(self,train_data,NewDict):
        #print(" in make dic ")
        for col in train_data.columns.values:
            if check_categorical(col):
                NewDict[col] = FillWithUniqueCat(train_data,col)
            else :
                NewDict[col] = FillWithUniqueContinuous(train_data,col)
        #print(self.NewDict)
        return NewDict
        
    
    
    def FindMeanSquareError(self,train_data ,  attribute,col):
        left  = train_data[train_data[col] == attribute]
        right = train_data[train_data[col] != attribute]
        mleft = left['SalePrice'].mean()
        mright= right['SalePrice'].mean()
        summ  = 0
        summ1 = 0
        
        for val in left['SalePrice']:
            summ=summ+(val-mleft)*(val-mleft)
        
        for val in right['SalePrice']:
            summ1=summ1+(val-mright)*(val-mright)
        countUp = left.shape[1]
        countDown = right.shape[1]
        over_all_mean =  ((countUp*summ)/(countUp + countDown))  +  ((countDown*summ1)/(countUp + countDown))
        #print(" mean = ")
        #print(over_all_mean)
        return [over_all_mean , col, attribute] 
            
    
    def FindUpAndDown(self,train_data,lis,col):
        #print(" in FindUpAndDown ")
        MSE = []
        i=0
        for attribute in lis:
                MSE.append(self.FindMeanSquareError(train_data,attribute,col))
        # MSE = [self.FindMeanSquareError(train_data, attribute, col) for attribute in lis].sort()[0]
        #print(MSE)    
        MSE.sort()
        #print(MSE[0])
        return MSE[0]
    

    def FindMeanSquareErrorNUm(self,train_data, left, right,col,x):
        meanl = left['SalePrice'].mean()
        meanr = left['SalePrice'].mean()
        summ =0
        for i in left['SalePrice']:
            summ = summ + (i -meanl)*(i - meanl)   
        summ1=0
        for i in right['SalePrice']:
            summ1 = summ1 + (i -meanr)*(i - meanr)
        total = len(left) + len(right)
        overAllMean = ((summ*len(left))/total) + ((summ1*len(right))/total)
        #print(" mean num = ")
        #print(overAllMean)
        
        return [overAllMean,col, x] 
            
    
    def FindUpAndDownForNUm(self,train_data, lis, col):
        #print(" in FindUpAndDownForNUm ")

        MSE = []
        i=0
       # print(" lis = ")
        #print(len(lis))
        for i in range(len(lis)):
            left = train_data[train_data[col] <= lis[i]]
            right= train_data[train_data[col] > lis[i]]
            MSE.append(self.FindMeanSquareErrorNUm(train_data, left, right,col,lis[i]))
            i=i+1
            
        MSE.sort()
        #print(len(MSE))
        #print(MSE[0])
        #print(MSE)
        return MSE[0]
    ## alright*****************************        
    def check_purity(self,data):
        label_column = data['SalePrice'].values
        unique_classes = np.unique(label_column)
        if len(unique_classes) == 1:
            return True
        return False
        
    def make_split(self,train_data):
       # print("in make split")
        val = []
        NewDict = dict()
        NewDict = self.make_dictionary(train_data,NewDict )
        #print(" dict size = ")
        #print(len(NewDict))
        column = train_data.columns.values
        #print(column)
        for col in column:
            if col == 'SalePrice':
                continue
            if check_categorical(col):
                lis = NewDict[col]
                val.append(self.FindUpAndDown(train_data , lis, col))
            else :
                lis = NewDict[col]
                val.append(self.FindUpAndDownForNUm(train_data , lis, col))
        val.sort()
        return val[0]
    
    
    def split_data(self,data , col , attribute):
        if check_categorical(col):
            data_below = data[data[col] == attribute]
            data_above = data[data[col] != attribute]
        else:
            data_below = data[data[col] <= attribute]
            data_above = data[data[col] > attribute]
        return data_below, data_above
    
    #*********
    def classify_data(self,data):
        label_column = data['SalePrice']
        unique_classes, counts_unique_classes = np.unique(
            label_column,return_counts = True)
        index = counts_unique_classes.argmax()
        classification = unique_classes[index]
        return classification
    
    
    
    def DecisionTree(self,train_data, counter = 0,
                     min_samples = 2, max_depth = 5):
        print(" IN decision tree ")
        if counter == 0:
            global col 
            col = train_data.columns
            data = train_data
        else:
            data = train_data
            
        if (self.check_purity(data)) or (data.shape[1] < min_samples) or (counter == max_depth):
            classification = self.classify_data(data)
            return classification
        else :
            counter+=1
            print(" data ")
            print(data.shape)
            mean, c , attribute = self.make_split(data)
            print(" mean c attribute ")
            print(mean)
            print(c)
            print(attribute)
            data_below, data_above = self.split_data(data, c, attribute)
            feature_name = c
            question = "{} <= {}".format(feature_name,attribute)
            sub_tree = {question : []}
            yes_answer = self.DecisionTree(data_below, counter, 
                                      min_samples,max_depth)
            no_answer = self.DecisionTree(data_above, counter, 
                                     min_samples, max_depth)
            if yes_answer == no_answer:
                self.sub_tree = yes_answer
            else :
                sub_tree[question].append(yes_answer)
                sub_tree[question].append(no_answer)

            return sub_tree
    
    
    def classify_example(self, example, tree):
        question = list(tree.keys())[0]
        feature_name, comparison_operator, value = question.split(" ")
        if(check_categorical(feature_name)):
            if example[feature_name] == value:
                answer = tree[question][0]
            else:
                answer = tree[question][1]

        else:
            if example[feature_name] <=float(value):
                answer = tree[question][0]
            else :
                answer = tree[question][1]

        if not isinstance(answer, dict):
            return answer

        else:
            residual_tree = answer
            return self.classify_example(example, residual_tree)
    

   

In [11]:
tree = dtree_regressor()
train_data = tree.train("train.csv")
print(tree.sub_tree)

 IN decision tree 
 data 
(1000, 76)
 mean c attribute 
2174739774483.6956
ExterQual
TA
 IN decision tree 
 data 
(616, 76)
 mean c attribute 
468520185524.20215
OverallQual
7
 IN decision tree 
 data 
(63, 76)
 mean c attribute 
61285917639.34426
LandSlope
Sev
 IN decision tree 
 data 
(2, 76)
 mean c attribute 
0.0
BedroomAbvGr
2
 IN decision tree 
 IN decision tree 
 IN decision tree 
 data 
(61, 76)
 mean c attribute 
47121565583.05556
GarageType
Detchd
 IN decision tree 
 data 
(16, 76)
 mean c attribute 
1926499487.1794868
BsmtFinType1
LwQ
 IN decision tree 
 IN decision tree 
 IN decision tree 
 data 
(45, 76)
 mean c attribute 
34432955388.396774
TotalBsmtSF
1352
 IN decision tree 
 IN decision tree 
 IN decision tree 
 data 
(553, 76)
 mean c attribute 
322962396673.79266
GarageFinish
Unf
 IN decision tree 
 data 
(375, 76)
 mean c attribute 
192389250291.23663
CentralAir
N
 IN decision tree 
 data 
(54, 76)
 mean c attribute 
13489694471.454548
BedroomAbvGr
4
 IN decision tre

In [23]:
predicted_price = tree.test('test.csv')


In [25]:
#predicted_price = [tree.classify_example(row, tree.sub_tree) for i, row in predicted_price.iterrows()]

In [26]:
predicted_price

[129000,
 135000,
 190000,
 135000,
 129000,
 135000,
 135000,
 39300,
 190000,
 135000,
 135000,
 190000,
 190000,
 135000,
 79000,
 315000,
 135000,
 190000,
 135000,
 135000,
 190000,
 190000,
 315000,
 135000,
 79000,
 135000,
 135000,
 190000,
 129000,
 135000,
 135000,
 190000,
 190000,
 135000,
 178400,
 135000,
 135000,
 190000,
 79000,
 135000,
 190000,
 135000,
 135000,
 135000,
 190000,
 135000,
 135000,
 190000,
 135000,
 135000,
 190000,
 190000,
 135000,
 135000,
 135000,
 135000,
 135000,
 39300,
 315000,
 135000,
 190000,
 129000,
 135000,
 190000,
 190000,
 129000,
 79000,
 135000,
 79000,
 129000,
 190000,
 129000,
 190000,
 129000,
 129000,
 129000,
 190000,
 190000,
 135000,
 190000,
 129000,
 315000,
 135000,
 135000,
 135000,
 190000,
 129000,
 190000,
 190000,
 129000,
 129000,
 135000,
 135000,
 135000,
 178400,
 135000,
 135000,
 135000,
 129000,
 135000,
 315000,
 190000,
 190000,
 190000,
 129000,
 190000,
 135000,
 190000,
 190000,
 190000,
 315000,
 190000,

In [27]:
label = pd.read_csv('test_labels.csv', header=None)

In [28]:
label[1].values

array([128000, 124900, 191000, 144000, 231500, 135750, 149700,  85000,
       196000, 155000, 146500, 171750, 385000, 139500, 136000, 446261,
       135000, 155000, 117500, 145000, 168500, 301000, 394617, 104900,
        80000, 135000, 120000, 164500, 143000, 115000, 157500, 272000,
       240000, 135000, 251000, 130250, 197500, 181134, 214500, 115000,
       213490, 124000, 172500, 160000, 200000, 150000, 142500, 149900,
       129500, 169000, 237000, 222500, 162500,  93500, 153500, 169000,
       144500,  60000, 325000, 143000, 305900, 197900,  96500, 187500,
       221000, 113000, 137000, 107000, 135900, 192000, 189000, 110000,
       241000, 180000, 128000, 149000, 136500, 424870,  62383, 314813,
       179900, 339750, 110000, 139000, 131000, 214000, 170000, 438780,
       317000, 185000, 175000, 188700, 381000, 110000, 227680, 134900,
       108480, 110000, 167900, 187500, 276000, 199900, 244000, 171900,
       184000, 173500, 117000, 144000, 225000, 239000, 239000, 235000,
      

In [30]:
from sklearn.metrics import mean_absolute_error

### Mean Absolute Error

In [31]:
mean_absolute_error(label[1].values, predicted_price)


40162.87391304348

### Mean Absolute Error With Sklearn Decision Tree :

In [32]:
import pandas as pd
import numpy as np
import matplotlib as plt
from pprint import pprint
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
%matplotlib inline
# %matplotlib notebook
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
TrainData  = pd.read_csv("train.csv")
TrainLabel = TrainData['SalePrice']
del TrainData['SalePrice']
TestData   = pd.read_csv("test.csv")
print(" Test Data ")
TestData.head()
print(" Train Data ")
TrainLabel.head()
#regression = DecisionTreeRegressor(random_state = 0)
#regression.fit(TrainData , TrainLabel)
#predicted_decision = regression.predict(TestData)



 Test Data 
 Train Data 


0    354000
1    378500
2    153500
3    173733
4    127000
Name: SalePrice, dtype: int64

### r2 Value

In [37]:
import pandas as pd
import numpy as np
import math
import operator
import sys
import matplotlib.pyplot as plt
import statistics 
from statistics import mode
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score, r2_score
from sklearn.metrics.pairwise import manhattan_distances
from sklearn.metrics import classification_report
import pandas as pd
r2_score(label[1].values, predicted_price)

0.37930356091795503